# HW 2: Language Modeling

In this homework you will be building several varieties of language models.

## Goal

We ask that you construct the following models in Torch / NamedTensor:

1. A count-based trigram model with linear-interpolation. $$p(y_t | y_{1:t-1}) =  \alpha_1 p(y_t | y_{t-2}, y_{t-1}) + \alpha_2 p(y_t | y_{t-1}) + (1 - \alpha_1 - \alpha_2) p(y_t) $$
2. A neural network language model (consult *A Neural Probabilistic Language Model* http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)
3. An LSTM language model (consult *Recurrent Neural Network Regularization*, https://arxiv.org/pdf/1409.2329.pdf) 
4. Your own extensions to these models.


Consult the papers provided for hyperparameters.

 


## Setup

This notebook provides a working definition of the setup of the problem itself. You may construct your models inline or use an external setup (preferred) to build your system.

In [1]:
!pip install -q torch torchtext opt_einsum
!pip install -qU git+https://github.com/harvardnlp/namedtensor

You are using pip version 19.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import torch
import torchtext
from torchtext.vocab import Vectors

from namedtensor import ntorch
from namedtensor.text import NamedField

import numpy as np

In [106]:
%run models/utils.py

The dataset we will use of this problem is known as the Penn Treebank (http://aclweb.org/anthology/J93-2004). It is the most famous dataset in NLP and includes a large set of different types of annotations. We will be using it here in a simple case as just a language modeling dataset.

To start, `torchtext` requires that we define a mapping from the raw text data to featurized indices. These fields make it easy to map back and forth between readable data and math, which helps for debugging.

In [3]:
# Our input $x$
TEXT = NamedField(names=("seqlen",))

Next we input our data. Here we will use the first 10k sentences of the standard PTB language modeling split, and tell it the fields.

In [5]:
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/input.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.5k.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/train.txt
!curl -qO https://raw.githubusercontent.com/harvard-ml-courses/cs287-s18/master/HW2/valid.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  185k  100  185k    0     0   965k      0 --:--:-- --:--:-- --:--:-- 1110k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  595k  100  595k    0     0  2595k      0 --:--:-- --:--:-- --:--:-- 2743k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4982k  100 4982k    0     0  3763k      0  0:00:01  0:00:01 --:--:-- 3794k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  390k  100  390k    0     0  1844k      0 --:--:-- --:--:-- --:--:-- 2012k


In [6]:
# Data distributed with the assignment
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(
    path=".", 
    train="train.txt", validation="valid.txt", test="valid.txt", text_field=TEXT)

The data format for language modeling is strange. We pretend the entire corpus is one long sentence.

In [7]:
print('len(train)', len(train))

len(train) 1


Here's the vocab itself. (This dataset has unk symbols already, but torchtext adds its own.)

In [8]:
TEXT.build_vocab(train)
print('len(TEXT.vocab)', len(TEXT.vocab))

len(TEXT.vocab) 10001


When debugging you may want to use a smaller vocab size. This will run much faster.

In [9]:
if False:
    TEXT.build_vocab(train, max_size=1000)
    len(TEXT.vocab)

The batching is done in a strange way for language modeling. Each element of the batch consists of `bptt_len` words in order. This makes it easy to run recurrent models like RNNs. 

In [10]:
from torchtext.data.iterator import BPTTIterator
from torchtext.data import Batch, Dataset
import math
 

class NamedBpttIterator(BPTTIterator):
    def __iter__(self):
        text = self.dataset[0].text
        TEXT = self.dataset.fields['text']
        TEXT.eos_token = None
        text = text + ([TEXT.pad_token] * int(math.ceil(len(text) / self.batch_size)
                                              * self.batch_size - len(text)))
        data = TEXT.numericalize(
            [text], device=self.device)
        data = (data
            .stack(("seqlen", "batch"), "flat")
            .split("flat", ("batch", "seqlen"), batch=self.batch_size)
            .transpose("seqlen", "batch")
        )

        dataset = Dataset(examples=self.dataset.examples, fields=[
            ('text', TEXT), ('target', TEXT)])
        while True:
            for i in range(0, len(self) * self.bptt_len, self.bptt_len):
                self.iterations += 1
                seq_len = min(self.bptt_len, len(data) - i - 1)
                yield Batch.fromvars(
                    dataset, self.batch_size,
                    text = data.narrow("seqlen", i, seq_len),
                    target = data.narrow("seqlen", i+1, seq_len),
                )
                         
            if not self.repeat:
                return

In [13]:
train_iter, val_iter, test_iter = NamedBpttIterator.splits(
    (train, val, test), batch_size=10, bptt_len=32, repeat=False)

Here's what these batches look like. Each is a string of length 32. Sentences are ended with a special `<eos>` token.

In [19]:
from namedtensor.nn import nn as nnn

In [18]:
TEXT.vocab.vectors

In [31]:
embedding_dim = 100
hidden_dim = 150
pad_idx = TEXT.vocab.stoi['<pad>']

embed = nnn.Embedding(num_embeddings=len(TEXT.vocab), embedding_dim=embedding_dim, padding_idx=pad_idx)

In [29]:
embed(batch.text).shape

OrderedDict([('seqlen', 32), ('batch', 10), ('embedding', 100)])

In [35]:
lstm = nnn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=1)

In [42]:
lstm = lstm.spec("embedding", "seqlen")

In [46]:
v1, v2 = lstm(embed(batch.text))

In [47]:
v1.shape

OrderedDict([('batch', 10), ('seqlen', 32), ('embedding', 150)])

In [ ]:
nnn.Linear(in_features=hidden_dim, out_features=len(TEXT.vocab)).spec("embedding", "vocab")

In [66]:
class LSTM(nnn.Module):
    def __init__(self, TEXT, embedding_dim=100, hidden_dim=150):
        super().__init__()
        
        pad_idx = TEXT.vocab.stoi['<pad>']
        
        self.embed = nnn.Embedding(num_embeddings=len(TEXT.vocab), 
                                   embedding_dim=embedding_dim, 
                                   padding_idx=pad_idx)
        
        self.lstm = nnn.LSTM(input_size=embedding_dim, 
                             hidden_size=hidden_dim, 
                             num_layers=1) \
                        .spec("embedding", "seqlen")
        
        self.w = nnn.Linear(in_features=hidden_dim, 
                            out_features=len(TEXT.vocab)) \
                        .spec("embedding", "vocab")
    
    def forward(self, batch_text):
        embedded = self.embed(batch_text)
        hidden_states, _ = self.lstm(embedded)
        log_probs = self.w(hidden_states)
        
        return log_probs

In [79]:
lstm = LSTM(TEXT)

In [80]:
lstm_out = lstm(batch.text)

In [109]:
lstm_loss = nnn.CrossEntropyLoss().spec('batch')

optim = torch.optim.Adam(lstm.parameters(), lr=1e-3)

In [110]:
def cb(**kwargs):
    ep, t, v = (kwargs['epoch'], (kwargs['train_loss'] / len(train_iter)), (kwargs['val_loss'] / len(val_iter)))
    print(f'Epoch {ep}, train loss = {t:.2f} [{np.exp(t):.2f}], val loss = {v:.2f} [{np.exp(v):.2f}]')
    
train_model(lstm, lstm_loss, optim, 
            train_iter, val_iter, num_epochs=2, 
            callback=cb,
            progress_bar=True)

AttributeError: 'LSTM' object has no attribute '_schema'

In [107]:
loss(lstm(batch.text), batch.target).exp()

NamedTensor(
	tensor(10121.6426, grad_fn=<ExpBackward>),
	())

In [14]:
it = iter(train_iter)
batch = next(it) 
print("Size of text batch [max bptt length, batch size]", batch.text.shape)
example = batch.text[{"batch": 1}]
print("Second in batch", example)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in example.values.data]))

Size of text batch [max bptt length, batch size] OrderedDict([('seqlen', 32), ('batch', 10)])
Second in batch NamedTensor(
	tensor([  72,    5,   28,  247,   61,   12,  216,    5,    0, 1847,   10,    4,
          72,  547,    3, 6506,  163,    7,  105,  479,   38,   31,  295, 4901,
          13,    4,   49,    3,    0,    0,   25, 2471]),
	('seqlen',))
Converted back to string:  shares of its common stock for each of <unk> deposit 's N shares outstanding <eos> liberty national a bank holding company has assets exceeding $ N billion <eos> <unk> <unk> was appointed


The next batch will be the continuation of the previous. This is helpful for running recurrent neural networks where you remember the current state when transitioning.

In [ ]:
batch = next(it)
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.text[{"batch": 1}].values.data]))

The batch object also contains the targets for the given batch in the field `target`. The target is simply the text offset by one.

In [ ]:
print("Converted back to string: ", " ".join([TEXT.vocab.itos[i] for i in batch.target.get("batch", 1).values.data]))

## Assignment

Now it is your turn to build the models described at the top of the assignment. 

Using the data given by this iterator, you should construct 3 different torch models that take in batch.text and produce a distribution over the next word. 

When a model is trained, use the following test function to produce predictions, and then upload to the kaggle competition: https://www.kaggle.com/c/harvard-cs287-s19-hw2/

For the final Kaggle test, we will have you do a next word prediction task. We will provide a 10 word prefix of sentences, and it is your job to predict 20 possible next word candidates

In [ ]:
!head input.txt

As a sample Kaggle submission, let us build a simple unigram model.  

In [ ]:
from collections import Counter
count = Counter()
for b in iter(train_iter):
    count.update(b.text.values.contiguous().view(-1).tolist())
count[TEXT.vocab.stoi["<eos>"]] = 0
predictions = [TEXT.vocab.itos[i] for i, c in count.most_common(20)]
with open("sample.txt", "w") as fout:
    print("id,word", file=fout)
    for i, l in enumerate(open("input.txt"), 1):
        print("%d,%s"%(i, " ".join(predictions)), file=fout)


The metric we are using is mean average precision of your 20-best list. 

$$MAP@20 = \frac{1}{|D|} \sum_{u=1}^{|D|} \sum_{k=1}^{20} Precision(u, 1:k)$$

Ideally we would use log-likelihood or ppl as discussed in class, but this is the best Kaggle gives us. This takes into account whether you got the right answer and how highly you ranked it. 

In particular, we ask that you do not game this metric. Please submit *exactly 20* unique predictions for each example.


As always you should put up a 5-6 page write-up following the template provided in the repository: https://github.com/harvard-ml-courses/nlp-template